# Topic Modeling of Yelp Academy Dataset

***Yading Guo***

**2016/11/15**

Topic modeling was carried out in this report on Yelp academy dataset. There are two main parts of this report.
* **Topic modeling of all the reviews in the dataset**
* **Comparison of topics of reviews from different sex**

## Topic modeling of all the reviews in the dataset

Three major steps are involved in this part. 
* Processing the raw reviews into words
* Constructing TDIDF vector used for topic modeling
* Fitting topic model 

### Firstly, processing the raw reviews into words

In the beginning, I import some modules for later usage. 
* nltk and string module are used for natural languange processing
* json and io are used for file read and write
* graphlab is used for topic modeling
* pyLDAvis is used for topic visualization

In [2]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import json
import graphlab
import io
import sexmachine.detector as gender
import pyLDAvis
import pyLDAvis.graphlab

This non-commercial license of GraphLab Create for academic use is assigned to adenguo@gmail.com and will expire on August 05, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1479187159.log


Some constants were defined and loaded. They are raw data, hyperparameter of constructing TDIDF model, stopwords list and so on.

And some functions are defined.

In [3]:
male_id = set()
female_id = set()
data_user = []
data_review = []    
with open('data/yelp_academic_dataset_review.json') as f:
    for line in f:
        data_review.append(json.loads(line))
with open('data/yelp_academic_dataset_user.json') as f:
    for line in f:
        data_user.append(json.loads(line))
trim_value = 2
min_length = 10
extra_words = set(["food", "it", "get", "go", "u"])
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
detector1 = gender.Detector()
pyLDAvis.enable_notebook()
def write_text_lists(texts_list, file_name):
    f = io.open(file_name, 'w+', encoding='utf8')
    for line in texts_list:
        f.write(u','.join(line) + '\n')
    f.close()
def load_text_lists(file_name):
    f = io.open(file_name, 'r', encoding='utf8')
    lines = f.readlines()
    return [line.strip().split(',') for line in lines]
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
def data_review_to_sf_bag_words(texts_list):
    sf_text = graphlab.SFrame({'text': texts_list})
    encoder = graphlab.feature_engineering.WordCounter()
    transformed_sf = encoder.fit_transform(sf_text)
    return transformed_sf
def data_review_to_sf_tfidf(texts_list):
    sf_text = graphlab.SFrame({'text': texts_list})
    encoder = graphlab.feature_engineering.WordCounter()
    bag_words_sf = encoder.fit_transform(sf_text)
    encoder_tfidf = graphlab.feature_engineering.TFIDF('text')
    encoder_tfidf = encoder_tfidf.fit(bag_words_sf)
    result = encoder_tfidf.transform(bag_words_sf)
    return result
def create_topic_model(text_file):
    print "loading save text"
    text_list = load_text_lists(text_file)
    print "creating bag of words vector"
    bag_of_words_vector = data_review_to_sf_bag_words(text_list)
    print "triming vector by value "+ str(trim_value)
    bag_of_words_vector = bag_of_words_vector['text'].dict_trim_by_values(trim_value)
    print "delect short line then " + str(min_length)
    ix = bag_of_words_vector.apply(lambda x: len(x.keys()) >= min_length)
    bag_of_words_vector = bag_of_words_vector[ix]
    print "remove extra words"
    bag_of_words_vector = bag_of_words_vector.dict_trim_by_keys(extra_words,exclude=True)
    print "creating tfidf vector"
    tfidf_vector = graphlab.text_analytics.tf_idf(bag_of_words_vector)
    model = graphlab.topic_model.create(tfidf_vector,
                              num_topics=10,       # number of topics
                              num_iterations=100,   # algorithm parameters
                              alpha=10, beta=0.1)  # hyperparameters
    return model,tfidf_vector
def split_male_female(data_user, data_review):
    for user in data_user:
        if detector1.get_gender(user['name']) == "male":
            male_id.add(user['user_id'])
        elif detector1.get_gender(user['name']) == "female":
            female_id.add(user['user_id'])
        else:
            pass
    male_review = []
    female_review = []
    for review in data_review:
        if review['user_id'] in male_id:
            male_review.append(review['text'])
        elif review['user_id'] in female_id:
            female_review.append(review['text'])
        else:
            pass
    return male_review,female_review

/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):


Following steps were done when the raw data is processed into list of words.
1. Stop words are removed.
2. Punctuations are removed.
3. Lemmatization are performed on each words. 
4. Tokenization the text into a list of words.

In the following funtions, the function *clean* is the workhorse of there steps. There processed data is then write into a file using function *write_text_lists*

In [4]:
text_data = [x['text'] for x in data_review]
text_list1 = [clean(doc).split() for doc in text_data]
write_text_lists(text_list1, 'clean_text.txt')

### Secondly, construction of TFIDF vector

There are two procedures in this part.
1. Removing very rare words which is the words appearing less than 2 times in the corpus. And delete very short reviews which is reviews which contain less than 10 unique words.
2. A vector of bag of words is construct and then it is converted into a vector of TFIDF.

The first part of function *creat_topic_model* is doing above two steps.

### Thirdly, Fitting topic model
The second part of funtion *creating_topic_model* is doing this part.
The interative visualization of the topic model is construct.

In [5]:
all_model,all_tfidf = create_topic_model('clean_text.txt')
pyLDAvis.graphlab.prepare(all_model, all_tfidf)

loading save text
creating bag of words vector
triming vector by value 2
delect short line then 10
remove extra words
creating tfidf vector


Learning a topic model

Number of documents    312809

Vocabulary size     53821

Running collapsed Gibbs sampling

...

| 90        | 41.82s        | 1.36636e+07    | 0               |

| 100       | 46.46s        | 1.29448e+07    | 0               |

+-----------+---------------+----------------+-----------------+

/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
/Users/adenguo/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      18.831817        1       1 -0.119276  0.061408
1      12.918530        1       2  0.129708  0.042846
8      12.316287        1       3  0.113702  0.124692
9      10.648200        1       4 -0.260421  0.058118
3       9.211395        1       5  0.089062  0.059357
2       7.585600        1       6 -0.017205 -0.083768
7       7.512844        1       7  0.029760 -0.066039
5       7.267659        1       8 -0.013236 -0.062835
4       7.225145        1       9  0.017616 -0.093316
6       6.482523        1      10  0.030289 -0.040463, topic_info=      Category           Freq        Term          Total  loglift  logprob
term                                                                      
6741   Default  289711.000000        room  289711.000000  30.0000  30.0000
39025  Default  188064.000000       hotel  188064.000000  29.0000  29.0000
10555  Default  104928.000000        thai  104928.000000  28.0000  28.0000
30410  Default  158155.000000       pizza  158155.000000  27.0000  27.0000
47785  Default  154577.000000        show  154577.000000  26.0000  26.0000
48906  Default  277490.000000        good  277490.000000  25.0000  25.0000
16135  Default  139902.000000        taco  139902.000000  24.0000  24.0000
53358  Default  175096.000000     chicken  175096.000000  23.0000  23.0000
2652   Default  128497.000000        rice  128497.000000  22.0000  22.0000
34006  Default  126246.000000         dog  126246.000000  21.0000  21.0000
52732  Default   88411.000000      noodle   88411.000000  20.0000  20.0000
47378  Default  155826.000000         bar  155826.000000  19.0000  19.0000
24185  Default   80923.000000         ice   80923.000000  18.0000  18.0000
24475  Default   90891.000000   chocolate   90891.000000  17.0000  17.0000
31165  Default  119613.000000         car  119613.000000  16.0000  16.0000
48718  Default  146411.000000      burger  146411.000000  15.0000  15.0000
32105  Default   59736.000000     cupcake   59736.000000  14.0000  14.0000
1602   Default   85064.000000       cream   85064.000000  13.0000  13.0000
34024  Default   93022.000000        cake   93022.000000  12.0000  12.0000
9090   Default  125596.000000       salad  125596.000000  11.0000  11.0000
38149  Default  273219.000000       place  273219.000000  10.0000  10.0000
30121  Default  104417.000000        pool  104417.000000   9.0000   9.0000
1909   Default  143505.000000       sauce  143505.000000   8.0000   8.0000
7903   Default   66744.000000         pho   66744.000000   7.0000   7.0000
34768  Default  122025.000000      cheese  122025.000000   6.0000   6.0000
8045   Default  115642.000000      flavor  115642.000000   5.0000   5.0000
24034  Default   90501.000000        beer   90501.000000   4.0000   4.0000
5734   Default  147811.000000     ordered  147811.000000   3.0000   3.0000
29967  Default  156464.000000       drink  156464.000000   2.0000   2.0000
51087  Default   62589.000000       curry   62589.000000   1.0000   1.0000
...        ...            ...         ...            ...      ...      ...
16135  Topic10   15430.503865        taco  139902.989100   0.5315  -5.6114
18135  Topic10    8624.651126     burrito   59202.030287   0.8097  -6.1931
40002  Topic10    2922.283569        exam    9523.033103   1.5547  -7.2753
2652   Topic10   12329.648858        rice  128497.572203   0.3922  -5.8357
36630  Topic10    9208.244779       staff   79516.929042   0.5802  -6.1276
15181  Topic10    8014.343679     mexican   67350.098180   0.6074  -6.2665
6741   Topic10   17347.732309        room  289711.570325  -0.0794  -5.4942
38149  Topic10   16720.985604       place  273219.389330  -0.0576  -5.5310
15226  Topic10   10219.259135      always  106887.414014   0.3886  -6.0234
48906  Topic10   16499.055623        good  277490.203860  -0.0864  -5.5444
6559   Topic10    9500.041605        star  121327.147141   0.1889  -6.0964
47785  To

As we can see from the visualization. There are several apparent topic. For example, topic 8 is about foreign cuisine. Topic 3 is about shops providing service other than food, such as salon, car service and so on. Topic 1 is about ordinary food. 

## Comparison of topics of reviews from different sex

I split the reviews into reviews from male and reviews from female.

This is done by function *split_male_female*. 


Then I do exactly the same procedures as the first part of this report on both reviews to produce male topic model and female topic model and visualized them to compare.

In [6]:
male_review, female_review = split_male_female(data_user, data_review)
male_text_list = [clean(doc).split() for doc in male_review]
female_text_list = [clean(doc).split() for doc in female_review]
write_text_lists(male_text_list, 'male_clean_text.txt')
write_text_lists(female_text_list, 'female_clean_text.txt')

In [7]:
male_model,male_tfidf = create_topic_model('male_clean_text.txt')
female_model,female_tfidf = create_topic_model('female_clean_text.txt')

loading save text
creating bag of words vector
triming vector by value 2
delect short line then 10
remove extra words
creating tfidf vector


Learning a topic model

Number of documents    100570

Vocabulary size     33456

Running collapsed Gibbs sampling

...

| 70        | 16.36s        | 1.0747e+07     | 0               |

| 80        | 18.70s        | 1.06371e+07    | 0               |

| 90        | 21.02s        | 1.01687e+07    | 0               |

| 100       | 23.24s        | 1.19234e+07    | 0               |

+-----------+---------------+----------------+-----------------+

In [8]:
pyLDAvis.graphlab.prepare(male_model, male_tfidf)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      18.770847        1       1 -0.101857 -0.092230
8      12.959471        1       2  0.020258 -0.141825
2      11.400524        1       3  0.007736  0.130231
7       9.906083        1       4  0.018273  0.085835
1       9.892811        1       5  0.052817 -0.111348
3       8.793875        1       6  0.059438  0.071037
9       7.614853        1       7 -0.253529  0.033558
6       7.469746        1       8  0.049193  0.039525
5       6.869404        1       9  0.068800 -0.028891
4       6.322387        1      10  0.078873  0.014108, topic_info=      Category          Freq      Term         Total  loglift  logprob
term                                                                  
4146   Default  98313.000000      room  98313.000000  30.0000  30.0000
7754   Default  44635.000000      taco  44635.000000  29.0000  29.0000
29673  Default  49695.000000      show  49695.000000  28.0000  28.0000
24240  Default  64189.000000     hotel  64189.000000  27.0000  27.0000
11589  Default  47478.000000     store  47478.000000  26.0000  26.0000
19408  Default  43647.000000       car  43647.000000  25.0000  25.0000
9384   Default  23109.000000   mexican  23109.000000  24.0000  24.0000
6165   Default  20733.000000     salsa  20733.000000  23.0000  23.0000
27702  Default  32703.000000      club  32703.000000  22.0000  22.0000
11204  Default  21628.000000   burrito  21628.000000  21.0000  21.0000
18659  Default  51384.000000     drink  51384.000000  20.0000  20.0000
30251  Default  48093.000000    burger  48093.000000  19.0000  19.0000
23698  Default  91411.000000     place  91411.000000  18.0000  18.0000
29411  Default  55779.000000       bar  55779.000000  17.0000  17.0000
3058   Default  33416.000000      pool  33416.000000  16.0000  16.0000
18925  Default  55670.000000     pizza  55670.000000  15.0000  15.0000
31020  Default  17699.000000      bean  17699.000000  14.0000  14.0000
2670   Default  19773.000000      chip  19773.000000  13.0000  13.0000
4025   Default  26365.000000      stay  26365.000000  12.0000  12.0000
14950  Default  37337.000000      beer  37337.000000  11.0000  11.0000
30370  Default  83339.000000      good  83339.000000  10.0000  10.0000
21674  Default  29481.000000      shop  29481.000000   9.0000   9.0000
773    Default  36526.000000    buffet  36526.000000   8.0000   8.0000
6491   Default  28924.000000      thai  28924.000000   7.0000   7.0000
26961  Default   6856.000000       und   6856.000000   6.0000   6.0000
18339  Default  35861.000000    casino  35861.000000   5.0000   5.0000
24042  Default  41622.000000      dish  41622.000000   4.0000   4.0000
11508  Default   7316.000000       die   7316.000000   3.0000   3.0000
2464   Default  47136.000000     night  47136.000000   2.0000   2.0000
33148  Default  42940.000000   chicken  42940.000000   1.0000   1.0000
...        ...           ...       ...           ...      ...      ...
9016   Topic10   1923.703913        un   2074.175193   2.6858  -6.5483
13623  Topic10   2600.714214       den   2922.988481   2.6443  -6.2468
13622  Topic10   1123.163297       dem   1173.664348   2.7171  -7.0864
14800  Topic10   1347.763871         à   1448.988734   2.6887  -6.9041
1411   Topic10   1188.939179       von   1264.809301   2.6992  -7.0295
3538   Topic10   4488.963322         e   5782.061805   2.5079  -5.7009
33416  Topic10   1509.797142        et   1734.359841   2.6224  -6.7906
18704  Topic10   3609.812505  terminal   5165.019215   2.4028  -5.9189
5718   Topic10   1750.440613       que   2168.700375   2.5468  -6.6427
5671   Topic10   3860.081716   wedding   6664.148488   2.2150  -5.8519
10557  Topic10   4779.339778        de  10136.636475   2.0092  -5.6382
13584  Topic10   5122.657797   airport  12700.337788   1.8531  -5.5689
19290  Topic10   1647.766065       war   2223.920893   2.4612  -6.7031
15065  Topic10   3386.816221      tour   83

In [9]:
pyLDAvis.graphlab.prepare(female_model, female_tfidf)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      17.550794        1       1  0.115566 -0.003325
1      11.952652        1       2  0.045538  0.149989
2      11.512200        1       3 -0.139700 -0.101214
7      11.459935        1       4 -0.057845 -0.074864
8      10.099782        1       5  0.035044  0.069626
9       8.351141        1       6  0.241899 -0.098340
3       8.051065        1       7 -0.066916 -0.030192
4       7.662295        1       8 -0.051218 -0.012903
6       6.893229        1       9 -0.065014  0.057905
5       6.466909        1      10 -0.057354  0.043318, topic_info=      Category           Freq       Term          Total  loglift  logprob
term                                                                     
4177   Default  132545.000000       room  132545.000000  30.0000  30.0000
24738  Default   84129.000000      hotel   84129.000000  29.0000  29.0000
6068   Default   45028.000000       hair   45028.000000  28.0000  28.0000
6194   Default   39208.000000       nail   39208.000000  27.0000  27.0000
30373  Default   35579.000000    massage   35579.000000  26.0000  26.0000
3100   Default   52516.000000       pool   52516.000000  25.0000  25.0000
22424  Default   63214.000000      table   63214.000000  24.0000  24.0000
10745  Default   24204.000000         dr   24204.000000  23.0000  23.0000
11124  Default   29679.000000        spa   29679.000000  22.0000  22.0000
27784  Default   61956.000000      order   61956.000000  21.0000  21.0000
30138  Default   61684.000000      sushi   61684.000000  20.0000  20.0000
19259  Default   51558.000000      pizza   51558.000000  19.0000  19.0000
19724  Default   46116.000000        car   46116.000000  18.0000  18.0000
22094  Default   60259.000000       show   60259.000000  17.0000  17.0000
25267  Default   26956.000000      class   26956.000000  16.0000  16.0000
31014  Default   97960.000000       good   97960.000000  15.0000  15.0000
770    Default   44426.000000     buffet   44426.000000  14.0000  14.0000
512    Default   60174.000000       roll   60174.000000  13.0000  13.0000
18979  Default   64288.000000      drink   64288.000000  12.0000  12.0000
30897  Default   50126.000000     burger   50126.000000  11.0000  11.0000
11828  Default   53748.000000      store   53748.000000  10.0000  10.0000
24174  Default  101840.000000      place  101840.000000   9.0000   9.0000
15504  Default   42479.000000  chocolate   42479.000000   8.0000   8.0000
4605   Default   24957.000000      salon   24957.000000   7.0000   7.0000
30696  Default   16637.000000     doctor   16637.000000   6.0000   6.0000
27788  Default   23036.000000     office   23036.000000   5.0000   5.0000
4073   Default   36191.000000       stay   36191.000000   4.0000   4.0000
3565   Default   62887.000000    ordered   62887.000000   3.0000   3.0000
1191   Default   46606.000000      sauce   46606.000000   2.0000   2.0000
560    Default   52984.000000     minute   52984.000000   1.0000   1.0000
...        ...            ...        ...            ...      ...      ...
8970   Topic10    6457.374313       vega   60202.859281   0.5060  -5.5605
4177   Topic10    9757.512487       room  132545.571386   0.1296  -5.1477
24738  Topic10    7248.013308      hotel   84129.127076   0.2868  -5.4450
25292  Topic10    1152.204996      piano    2670.765207   1.8978  -7.2841
20154  Topic10    1194.396868        los    2965.477648   1.8291  -7.2481
30138  Topic10    5006.340796      sushi   61684.641473   0.2271  -5.8150
512    Topic10    4910.950477       roll   60174.115726   0.2327  -5.8343
30414  Topic10    4208.364084       soup   44253.851398   0.3856  -5.9887
21545  Topic10    4382.634860        dog   49100.753842   0.3222  -5.9481
3100   Topic10    4356.952851       pool   52516.216971   0.2491  -5.9540
6068   Topic10    3999.239152       hair   45028.576899   0.3173  -6.0396
24174  Topic10    5631.881162      place  101840.7606

As we can see, there are obvious difference existing between male's and female's topics. There is a topic, 3, for male is about car, repair and problem. Other topics from male are about Mexico food(topic 9), gambling(topic 4) and shows(topic 6). These topics don't show themselves in female's topics. Female's topics are concentrate on drink and dessert(topic 9), hair cut and massage(topic 8) and specific food(topic 5).